## Seleccionamos la Colección y definimos el rango en el que queremos trabajar

In [1]:
from collections import defaultdict
from pymongo import MongoClient
import time
import operator
import datetime
import re

client = MongoClient()
db = client.twitter
te = db['prueba2']
inicio =  datetime.datetime(2017, 4, 14,10)
fin = datetime.datetime(2017,4,14,19)
limit = 10



In [2]:
##AUX consultar por horas

request4days = te.aggregate([
    {"$match":
     {
         '$and': [{'created_at': {"$lt" : fin}} ,
                  {'created_at': {"$gt" : inicio}} ]
         
         
     }
        
        
    },
    
    
    {"$group": {
        "_id" : { "$concat": [
            {"$substr": [{"$year": "$created_at"}, 0, 4 ]},
            "-",
            {"$substr": [{"$month": "$created_at"}, 0, 2 ]},
            "-",
            {"$substr": [{"$dayOfMonth": "$created_at"}, 0, 2 ]},
            " ",
            {"$substr": [{"$hour": "$created_at"}, 0, 2 ]},
            
        ]},
        "count": {"$sum": 1 }
     }},
     {"$sort": { "_id": 1 }}
])

for a in request4days:
    print a["_id"], a["count"]

2017-4-14 10 3097
2017-4-14 11 3397
2017-4-14 12 3372
2017-4-14 13 3306
2017-4-14 14 3245
2017-4-14 15 3053
2017-4-14 16 3568
2017-4-14 17 3323
2017-4-14 18 3097


## Creamos un diccionario por URLS y metemos primera fecha

In [3]:
####Devolvemos un diccionario con las urls 
##entre dos periodos con la fecha mas antigua

def get_group_requests(inicio,fin):

    dw = defaultdict(lambda: [0, datetime.datetime.now()])
    for a in te.find({'$and': [{'created_at': {"$lt" : fin}} ,{'created_at': {"$gt" : inicio}} ]}):
        if len(a['entities']['urls']) >0:
            for b in a['entities']['urls']:
                if dw[b['expanded_url']]:
                    dw[b['expanded_url']][0] += 1
                    if(a['created_at'] < dw[b['expanded_url']][1]):
                        dw[b['expanded_url']][1]= a['created_at']
        if 'retweeted_status' in a and len(a['retweeted_status']['entities']['urls']) >0:      
            for b in a['retweeted_status']['entities']['urls']:
                    dw[b['expanded_url']][0] +=  1
                    if(a['created_at'] < dw[b['expanded_url']][1]):
                        dw[b['expanded_url']][1]= a['created_at']
    return dw

## Creamos lista ordenada a partir del diccionario 

In [4]:
#print dw
def sort_group_requests(dw, limit):
    lista = map (lambda (x,y): [x,y[0],y[1]],dw.items())
    trending = sorted(lista, key=operator.itemgetter(1), reverse=True)
#print trending
    i = 1
#trending = []
    urls = [] 
    medias = ['elpais', 'elmundo', 'eleconomista', 'elmundotoday', 'marca com', '20minutos', 'meneame', 'quora']
    for a in trending:
        if any(word in str(a[0]) for word in medias):
            urls.append(a)
            i = i+1
        if i > limit:
            break
    return urls

In [5]:
##Por grupos de X min 
from datetime import timedelta
print inicio
print fin
inicursor = inicio
endcursor = fin
nmins = 20
urls = {}
while inicursor<fin:
    endcursor = inicursor + timedelta(minutes=nmins)
    urls[inicursor] = sort_group_requests(
        get_group_requests(inicursor, endcursor), limit
    )
    inicursor = endcursor


2017-04-14 10:00:00
2017-04-14 19:00:00


In [6]:
for a in sorted(urls.keys()):
    print a 
    print len(urls[a])
    print '--'

2017-04-14 10:00:00
10
--
2017-04-14 10:20:00
10
--
2017-04-14 10:40:00
10
--
2017-04-14 11:00:00
10
--
2017-04-14 11:20:00
10
--
2017-04-14 11:40:00
10
--
2017-04-14 12:00:00
10
--
2017-04-14 12:20:00
10
--
2017-04-14 12:40:00
10
--
2017-04-14 13:00:00
10
--
2017-04-14 13:20:00
10
--
2017-04-14 13:40:00
10
--
2017-04-14 14:00:00
10
--
2017-04-14 14:20:00
10
--
2017-04-14 14:40:00
10
--
2017-04-14 15:00:00
10
--
2017-04-14 15:20:00
10
--
2017-04-14 15:40:00
10
--
2017-04-14 16:00:00
10
--
2017-04-14 16:20:00
10
--
2017-04-14 16:40:00
10
--
2017-04-14 17:00:00
10
--
2017-04-14 17:20:00
10
--
2017-04-14 17:40:00
10
--
2017-04-14 18:00:00
10
--
2017-04-14 18:20:00
10
--
2017-04-14 18:40:00
10
--


## Analizamos la info de las urls extraidas

In [7]:
import urllib2
from bs4 import BeautifulSoup

def get_data_pais(name):
    opener = urllib2.build_opener()
    opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
    webpage = opener.open(name).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    title = soup.find("meta",  property="og:title")
    title2 = soup.find("meta",  {"name":"title"})
    title3 = soup.find("title")
    url = soup.find("meta",  property="og:url")
    desc = soup.find("meta",  property="og:description")
    desc2 = soup.find("meta",  {"name":"description"})
    img = soup.find("meta",  property="og:image")
    d = {}
    if title3 :
            title3= str(title3).replace("<title>", "").replace("</title>", "")
    d['title'] = title["content"] if title else (title2["content"] if title2 else title3)
    d['url'] = url["content"] if url else name
    d["desc"] = desc["content"] if desc else (desc2["content"] if desc2 else None)
    d["img"] = img["content"] if img else None
    
    if not d["img"]:
        imgs = soup.findAll("div", {"class":"article-image"})
        for img in imgs:
            soup2 = BeautifulSoup(str(img), 'html.parser')
            i = soup2.findAll("img")
            if i:
                i = str(i)[str(i).index('src="')+5:]
                i = i[:i.index('"')]
                d["img"] = i
    
    ####buscamos Tags
    tags = soup.findAll("meta",  property="article:tag")
    d['tags'] = ','.join([tag['content'] for tag in tags])
    
    return d
  
lurls = []
#urls = [[r'http://m.20minutos.es/deportes/noticia/der-spiegel-cristiano-ronaldo-pago-euros-evitar-juicio-presunta-violacion-3012042/0/',4]]




    



In [8]:
def get_final_urls(urls):
    for url in urls:
        print url[0]
        d = get_data_pais(url[0])
        time.sleep(1)
        d['rank'] = url[1]
        lurls.append(d)

In [9]:
final_lurls = {}

for a in sorted(urls.keys()):
    lurls = []
    print '------'
    print a
    print '------'
    for url in urls[a]:
        print url[0] + ' -> ' + str(url[1])
        d = get_data_pais(url[0])
        time.sleep(1)
        d['rank'] = url[1]
        lurls.append(d)
    final_lurls[a] = lurls
    

------
2017-04-14 10:00:00
------
http://www.elmundo.es/cronica/2017/04/14/58e7bbf746163f324e8b45fd.html -> 46
http://www.elmundo.es/economia/macroeconomia/2017/04/14/58efb52c468aebde2f8b45e4.html -> 43
http://cultura.elpais.com/cultura/2017/04/13/actualidad/1492099247_028284.html?id_externo_rsoc=TW_CC -> 34
http://politica.elpais.com/politica/2017/04/13/actualidad/1492092111_475157.html?id_externo_rsoc=TW_CC -> 29
http://www.elmundo.es/andalucia/2017/04/14/58f04e36e2704eeb718b466a.html -> 28
http://internacional.elpais.com/internacional/2017/04/13/estados_unidos/1492102620_991509.html -> 17
http://tecnologia.elpais.com/tecnologia/2017/04/11/actualidad/1491925308_828582.html?id_externo_rsoc=TW_CC -> 13
http://cultura.elpais.com/cultura/2017/04/13/actualidad/1492107162_872898.html?id_externo_rsoc=TW_CC -> 12
http://www.meneame.net/story/gallardon-investigado-juzgado-44-madrid-malversacion-cuentas-m30 -> 12
http://tecnologia.elpais.com/tecnologia/2017/03/30/actualidad/1490875903_152817.h

http://economia.elpais.com/economia/2017/04/13/actualidad/1492038022_635203.html?id_externo_rsoc=TW_CC -> 25
http://www.20minutos.es/noticia/3011806/0/pedro-sanchez-critica-banderas-media-asta-sema-santa/?utm_source=Twitter&utm_medium=Social&utm_campaign=Mobile-web -> 25
http://www.elmundo.es/cronica/2017/04/14/58e7bbf746163f324e8b45fd.html -> 17
http://politica.elpais.com/politica/2017/04/13/actualidad/1492092111_475157.html?id_externo_rsoc=TW_CC -> 12
http://www.elmundo.es/internacional/2017/04/14/58f0a80622601d141d8b459c.html -> 10
http://internacional.elpais.com/internacional/2017/04/14/america/1492130889_914715.html?id_externo_rsoc=FB_CC -> 10
------
2017-04-14 13:00:00
------
http://internacional.elpais.com/internacional/2017/04/14/america/1492120887_809569.html?id_externo_rsoc=TW_CC -> 91
http://internacional.elpais.com/internacional/2017/04/14/america/1492120887_809569.html -> 52
http://internacional.elpais.com/internacional/2017/04/14/america/1492130889_914715.html?id_externo_

http://internacional.elpais.com/internacional/2017/04/14/america/1492120887_809569.html?id_externo_rsoc=TW_CC -> 13
http://www.elmundo.es/cronica/2017/04/14/58e7bbf746163f324e8b45fd.html -> 12
http://internacional.elpais.com/internacional/2017/04/13/mundo_global/1492097817_740242.html?id_externo_rsoc=TW_CC -> 11
http://www.elmundo.es/madrid/2017/04/13/58ee698de5fdea39558b45df.html -> 10
------
2017-04-14 15:40:00
------
http://cultura.elpais.com/cultura/2017/04/13/actualidad/1492107162_872898.html?id_externo_rsoc=TW_CM -> 49
http://internacional.elpais.com/internacional/2017/04/14/america/1492130889_914715.html?id_externo_rsoc=TW_CC -> 30
http://internacional.elpais.com/internacional/2017/04/14/actualidad/1492172234_286542.html -> 24
http://internacional.elpais.com/internacional/2017/04/14/america/1492120887_809569.html -> 22
http://politica.elpais.com/politica/2017/04/14/actualidad/1492180712_590970.html?id_externo_rsoc=TW_CC -> 18
http://cultura.elpais.com/cultura/2017/04/13/actualid

http://internacional.elpais.com/internacional/2017/04/13/estados_unidos/1492102620_991509.html -> 13
http://politica.elpais.com/politica/2017/04/12/actualidad/1492012423_170342.html -> 11
http://internacional.elpais.com/internacional/2017/04/14/america/1492120887_809569.html -> 10
http://internacional.elpais.com/internacional/2017/04/14/america/1492120887_809569.html?id_externo_rsoc=TW_CC -> 10
------
2017-04-14 18:20:00
------
http://www.elmundo.es/deportes/futbol/2017/04/14/58f0f266e2704e016f8b466c.html -> 52
http://elmundo.sv/gracias-el-paraiso-escondido-de-honduras/ -> 30
http://internacional.elpais.com/internacional/2017/04/14/america/1492130889_914715.html?id_externo_rsoc=TW_CC -> 19
http://www.elmundo.es/economia/macroeconomia/2017/04/14/58efb52c468aebde2f8b45e4.html -> 17
http://www.elmundo.es/andalucia/sevilla/2017/04/14/58f0fcf8268e3e0d458b45a8.html -> 15
http://www.elmundo.es/cultura/toros/2017/04/14/58f0109322601d75558b456e.html -> 13
http://elpais.com/elpais/2017/04/10/ico

## Volcamos a json

In [13]:
import json
final_lurls_json  = {}
for a in final_lurls.keys():
    final_lurls_json[a.strftime("%Y-%m-%d %H:%M")] = final_lurls[a] 
with open('tweevy', 'w') as f:
    f.write(json.dumps(final_lurls_json))

In [57]:
inter = sorted(final_lurls.keys())
import numpy as np
import pandas as pd
horas = []

for i in range (2, len(inter)):
    a = pd.DataFrame(final_lurls[inter[i-2]])
    

    b = pd.DataFrame(final_lurls[inter[i-1]])
    c = pd.DataFrame(final_lurls[inter[i]])
    hora = pd.concat([a, b, c])
    hora['inter'] = pd.Series(inter[i], index=hora.index)
    hora.sort_values(by='rank',ascending=False)
    horas.append(hora)




In [116]:
pd.options.display.max_colwidth = 90
pd.options.display.max_rows = 90

(horas[0])[['inter','title','desc', 'rank']].groupby(['inter','title','desc']).sum().sort_values(by='rank', ascending=False).head(5)



rank
inter               title                                                                                      desc                                                                                            
2017-04-14 10:40:00 La foto con Aznar del hijo del 'carnicero' de Siria                                        Septiembre de 2016. El ex presidente José María Aznar viste corbata verde y camisa blan...   103
                    Andalucía y Cataluña, donde más ha invertido el Estado desde 2008                          Las dos comunidades en las que más inversión ha presupuestado el Estado desde el estall...    85
                    Ocho detenidos y 17 heridos, uno de ellos grave, por las estampidas en la 'Madrugá' de ... Ocho detenidos y al menos 17 personas trasladadas al hospital, una de ellas en estado g...    82
                    La oposición venezolana recobra el impulso con la protesta en la calle                     Dos semanas después de las sentencias del Tribunal Supremo de Justicia son cinco los mu...    77
                    Tribuna | El monocultivo del cine español                                                  Por tercera vez, he recibido una negativa de TVE a una película mía                           63

In [117]:
horas[1][['inter','title','desc', 'rank']].groupby(['inter','title','desc']).sum().sort_values(by='rank', ascending=False).head(5)

rank
inter               title                                                                                      desc                                                                                            
2017-04-14 11:00:00 La oposición venezolana recobra el impulso con la protesta en la calle                     Dos semanas después de las sentencias del Tribunal Supremo de Justicia son cinco los mu...   121
                    La foto con Aznar del hijo del 'carnicero' de Siria                                        Septiembre de 2016. El ex presidente José María Aznar viste corbata verde y camisa blan...   111
                    Ocho detenidos y 17 heridos, uno de ellos grave, por las estampidas en la 'Madrugá' de ... Ocho detenidos y al menos 17 personas trasladadas al hospital, una de ellas en estado g...    74
                    Columna | Por los clavos de Cristo                                                         Dónde está el pecado en decir que parecen excesivos los eventos religiosos estos días         63
                    Andalucía y Cataluña, donde más ha invertido el Estado desde 2008                          Las dos comunidades en las que más inversión ha presupuestado el Estado desde el estall...    57

In [118]:
horas[2][['inter','title','desc', 'rank']].groupby(['inter','title','desc']).sum().sort_values(by='rank', ascending=False).head(5)

rank
inter               title                                                                                      desc                                                                                            
2017-04-14 11:20:00 La oposición venezolana recobra el impulso con la protesta en la calle                     Dos semanas después de las sentencias del Tribunal Supremo de Justicia son cinco los mu...   149
                    La foto con Aznar del hijo del 'carnicero' de Siria                                        Septiembre de 2016. El ex presidente José María Aznar viste corbata verde y camisa blan...   120
                    Columna | Por los clavos de Cristo                                                         Dónde está el pecado en decir que parecen excesivos los eventos religiosos estos días         71
                    Ocho detenidos y 17 heridos, uno de ellos grave, por las estampidas en la 'Madrugá' de ... Ocho detenidos y al menos 17 personas trasladadas al hospital, una de ellas en estado g...    57
                    Andalucía y Cataluña, donde más ha invertido el Estado desde 2008                          Las dos comunidades en las que más inversión ha presupuestado el Estado desde el estall...    47

In [119]:
horas[3][['inter','title','desc', 'rank']].groupby(['inter','title','desc']).sum().sort_values(by='rank', ascending=False).head(5)

rank
inter               title                                                                                      desc                                                                                            
2017-04-14 11:40:00 La oposición venezolana recobra el impulso con la protesta en la calle                     Dos semanas después de las sentencias del Tribunal Supremo de Justicia son cinco los mu...   113
                    La foto con Aznar del hijo del 'carnicero' de Siria                                        Septiembre de 2016. El ex presidente José María Aznar viste corbata verde y camisa blan...   112
                    Columna | Por los clavos de Cristo                                                         Dónde está el pecado en decir que parecen excesivos los eventos religiosos estos días         72
                    Ocho detenidos y 17 heridos, uno de ellos grave, por las estampidas en la 'Madrugá' de ... Ocho detenidos y al menos 17 personas trasladadas al hospital, una de ellas en estado g...    54
                    Andalucía y Cataluña, donde más ha invertido el Estado desde 2008                          Las dos comunidades en las que más inversión ha presupuestado el Estado desde el estall...    47

In [120]:
horas[4][['inter','title','desc', 'rank']].groupby(['inter','title','desc']).sum().sort_values(by='rank', ascending=False).head(5)

rank
inter               title                                                                                      desc                                                                                            
2017-04-14 12:00:00 La oposición venezolana recobra el impulso con la protesta en la calle                     Dos semanas después de las sentencias del Tribunal Supremo de Justicia son cinco los mu...   359
                    La foto con Aznar del hijo del 'carnicero' de Siria                                        Septiembre de 2016. El ex presidente José María Aznar viste corbata verde y camisa blan...    80
                    Andalucía y Cataluña, donde más ha invertido el Estado desde 2008                          Las dos comunidades en las que más inversión ha presupuestado el Estado desde el estall...    52
                    Columna | Por los clavos de Cristo                                                         Dónde está el pecado en decir que parecen excesivos los eventos religiosos estos días         50
                    El expresidente salvadoreño Funes, señalado por recibir 1,5 millones de dólares de Odeb... El exmandatario fue denunciado por un ex directivo de la constructora por recibir diner...    40

In [121]:
horas[5][['inter','title','desc', 'rank']].groupby(['inter','title','desc']).sum().sort_values(by='rank', ascending=False).head(5)

rank
inter               title                                                                                      desc                                                                                            
2017-04-14 12:20:00 La oposición venezolana recobra el impulso con la protesta en la calle                     Dos semanas después de las sentencias del Tribunal Supremo de Justicia son cinco los mu...   500
                    El expresidente salvadoreño Funes, señalado por recibir 1,5 millones de dólares de Odeb... El exmandatario fue denunciado por un ex directivo de la constructora por recibir diner...    71
                    La foto con Aznar del hijo del 'carnicero' de Siria                                        Septiembre de 2016. El ex presidente José María Aznar viste corbata verde y camisa blan...    69
                    Una estudiante británica asesinada al ser apuñalada en la Ciudad Vieja de Jerusalén        Un vagón del tranvía que cruza la disputada Jerusalén y es compartido diariamente por i...    39
                    Andalucía y Cataluña, donde más ha invertido el Estado desde 2008                          Las dos comunidades en las que más inversión ha presupuestado el Estado desde el estall...    38

In [122]:
horas[6][['inter','title','desc', 'rank']].groupby(['inter','title','desc']).sum().sort_values(by='rank', ascending=False).head(5)

rank
inter               title                                                                                      desc                                                                                            
2017-04-14 12:40:00 La oposición venezolana recobra el impulso con la protesta en la calle                     Dos semanas después de las sentencias del Tribunal Supremo de Justicia son cinco los mu...   616
                    El expresidente salvadoreño Funes, señalado por recibir 1,5 millones de dólares de Odeb... El exmandatario fue denunciado por un ex directivo de la constructora por recibir diner...   137
                    La foto con Aznar del hijo del 'carnicero' de Siria                                        Septiembre de 2016. El ex presidente José María Aznar viste corbata verde y camisa blan...    56
                    La Guardia Civil no ve acoso en gritar a una agente por salir a ponerse una compresa       El instituto armado archiva la queja en una resolución contra la que no cabe recurso          43
                    Una estudiante británica asesinada al ser apuñalada en la Ciudad Vieja de Jerusalén        Un vagón del tranvía que cruza la disputada Jerusalén y es compartido diariamente por i...    36

In [123]:
horas[7][['inter','title','desc', 'rank']].groupby(['inter','title','desc']).sum().sort_values(by='rank', ascending=False).head(5)

rank
inter               title                                                                                      desc                                                                                            
2017-04-14 13:00:00 La oposición venezolana recobra el impulso con la protesta en la calle                     Dos semanas después de las sentencias del Tribunal Supremo de Justicia son cinco los mu...   469
                    El expresidente salvadoreño Funes, señalado por recibir 1,5 millones de dólares de Odeb... El exmandatario fue denunciado por un ex directivo de la constructora por recibir diner...   159
                    La foto con Aznar del hijo del 'carnicero' de Siria                                        Septiembre de 2016. El ex presidente José María Aznar viste corbata verde y camisa blan...    54
                    La Guardia Civil no ve acoso en gritar a una agente por salir a ponerse una compresa       El instituto armado archiva la queja en una resolución contra la que no cabe recurso          36
                    Ocho detenidos y 17 heridos, uno de ellos grave, por las estampidas en la 'Madrugá' de ... Ocho detenidos y al menos 17 personas trasladadas al hospital, una de ellas en estado g...    34

In [124]:
horas[8][['inter','title','desc', 'rank']].groupby(['inter','title','desc']).sum().sort_values(by='rank', ascending=False).head(5)

rank
inter               title                                                                                      desc                                                                                            
2017-04-14 13:20:00 La oposición venezolana recobra el impulso con la protesta en la calle                     Dos semanas después de las sentencias del Tribunal Supremo de Justicia son cinco los mu...   482
                    El expresidente salvadoreño Funes, señalado por recibir 1,5 millones de dólares de Odeb... El exmandatario fue denunciado por un ex directivo de la constructora por recibir diner...   259
                    La foto con Aznar del hijo del 'carnicero' de Siria                                        Septiembre de 2016. El ex presidente José María Aznar viste corbata verde y camisa blan...    79
                    Secun de la Rosa                                                                           Escribe y dirige 'Mi hermano es un moderno' (Teatro Lara), un musical sobre la primera ...    45
                    Espías británicos alertaron de los contactos entre el equipo de Trump y Rusia              La CIA llama a WikiLeaks un  servicio de inteligencia hostil  alentado por Moscú              27

In [125]:
horas[9][['inter','title','desc', 'rank']].groupby(['inter','title','desc']).sum().sort_values(by='rank', ascending=False).head(5)

rank
inter               title                                                                                      desc                                                                                            
2017-04-14 13:40:00 La oposición venezolana recobra el impulso con la protesta en la calle                     Dos semanas después de las sentencias del Tribunal Supremo de Justicia son cinco los mu...   477
                    El expresidente salvadoreño Funes, señalado por recibir 1,5 millones de dólares de Odeb... El exmandatario fue denunciado por un ex directivo de la constructora por recibir diner...   237
                    La foto con Aznar del hijo del 'carnicero' de Siria                                        Septiembre de 2016. El ex presidente José María Aznar viste corbata verde y camisa blan...    73
                    Secun de la Rosa                                                                           Escribe y dirige 'Mi hermano es un moderno' (Teatro Lara), un musical sobre la primera ...    45
                    Libros que abordan de forma directa el acoso escolar                                       La literatura infantil puede ayudar a los niños a poner cara y combatir el acoso escolar      24

In [126]:
horas[10][['inter','title','desc', 'rank']].groupby(['inter','title','desc']).sum().sort_values(by='rank', ascending=False).head(5)

rank
inter               title                                                                                      desc                                                                                            
2017-04-14 14:00:00 La oposición venezolana recobra el impulso con la protesta en la calle                     Dos semanas después de las sentencias del Tribunal Supremo de Justicia son cinco los mu...   449
                    El expresidente salvadoreño Funes, señalado por recibir 1,5 millones de dólares de Odeb... El exmandatario fue denunciado por un ex directivo de la constructora por recibir diner...   260
                    La foto con Aznar del hijo del 'carnicero' de Siria                                        Septiembre de 2016. El ex presidente José María Aznar viste corbata verde y camisa blan...    75
                    Secun de la Rosa                                                                           Escribe y dirige 'Mi hermano es un moderno' (Teatro Lara), un musical sobre la primera ...    45
                    Libros que abordan de forma directa el acoso escolar                                       La literatura infantil puede ayudar a los niños a poner cara y combatir el acoso escolar      33

In [127]:
horas[11][['inter','title','desc', 'rank']].groupby(['inter','title','desc']).sum().sort_values(by='rank', ascending=False).head(5)

rank
inter               title                                                                                      desc                                                                                            
2017-04-14 14:20:00 La oposición venezolana recobra el impulso con la protesta en la calle                     Dos semanas después de las sentencias del Tribunal Supremo de Justicia son cinco los mu...   371
                    El expresidente salvadoreño Funes, señalado por recibir 1,5 millones de dólares de Odeb... El exmandatario fue denunciado por un ex directivo de la constructora por recibir diner...   190
                    Tribuna | El monocultivo del cine español                                                  Por tercera vez, he recibido una negativa de TVE a una película mía                           42
                    La foto con Aznar del hijo del 'carnicero' de Siria                                        Septiembre de 2016. El ex presidente José María Aznar viste corbata verde y camisa blan...    33
                    Libros que abordan de forma directa el acoso escolar                                       La literatura infantil puede ayudar a los niños a poner cara y combatir el acoso escolar      33

In [128]:
horas[12][['inter','title','desc', 'rank']].groupby(['inter','title','desc']).sum().sort_values(by='rank', ascending=False).head(5)

rank
inter               title                                                                                      desc                                                                                            
2017-04-14 14:40:00 La oposición venezolana recobra el impulso con la protesta en la calle                     Dos semanas después de las sentencias del Tribunal Supremo de Justicia son cinco los mu...   284
                    El expresidente salvadoreño Funes, señalado por recibir 1,5 millones de dólares de Odeb... El exmandatario fue denunciado por un ex directivo de la constructora por recibir diner...   186
                    La foto con Aznar del hijo del 'carnicero' de Siria                                        Septiembre de 2016. El ex presidente José María Aznar viste corbata verde y camisa blan...    42
                    Tribuna | El monocultivo del cine español                                                  Por tercera vez, he recibido una negativa de TVE a una película mía                           42
                    Columna | Por los clavos de Cristo                                                         Dónde está el pecado en decir que parecen excesivos los eventos religiosos estos días         35

In [129]:
horas[13][['inter','title','desc', 'rank']].groupby(['inter','title','desc']).sum().sort_values(by='rank', ascending=False).head(5)

rank
inter               title                                                                                      desc                                                                                            
2017-04-14 15:00:00 La oposición venezolana recobra el impulso con la protesta en la calle                     Dos semanas después de las sentencias del Tribunal Supremo de Justicia son cinco los mu...   237
                    El expresidente salvadoreño Funes, señalado por recibir 1,5 millones de dólares de Odeb... El exmandatario fue denunciado por un ex directivo de la constructora por recibir diner...   165
                    Columna | Por los clavos de Cristo                                                         Dónde está el pecado en decir que parecen excesivos los eventos religiosos estos días         36
                    El pasajero evacuado por United Airlines demandará a la compañía                           No pueden tratarnos como ganado , afirma el abogado de David Dao, que perdió dos diente...    34
                    La foto con Aznar del hijo del 'carnicero' de Siria                                        Septiembre de 2016. El ex presidente José María Aznar viste corbata verde y camisa blan...    33

In [130]:
horas[14][['inter','title','desc', 'rank']].groupby(['inter','title','desc']).sum().sort_values(by='rank', ascending=False).head(5)

rank
inter               title                                                                                      desc                                                                                            
2017-04-14 15:20:00 La oposición venezolana recobra el impulso con la protesta en la calle                     Dos semanas después de las sentencias del Tribunal Supremo de Justicia son cinco los mu...   174
                    El expresidente salvadoreño Funes, señalado por recibir 1,5 millones de dólares de Odeb... El exmandatario fue denunciado por un ex directivo de la constructora por recibir diner...   159
                    La foto con Aznar del hijo del 'carnicero' de Siria                                        Septiembre de 2016. El ex presidente José María Aznar viste corbata verde y camisa blan...    45
                    Columna | Por los clavos de Cristo                                                         Dónde está el pecado en decir que parecen excesivos los eventos religiosos estos días         40
                    El pasajero evacuado por United Airlines demandará a la compañía                           No pueden tratarnos como ganado , afirma el abogado de David Dao, que perdió dos diente...    34

In [131]:
horas[15][['inter','title','desc', 'rank']].groupby(['inter','title','desc']).sum().sort_values(by='rank', ascending=False).head(5)

rank
inter               title                                                                                      desc                                                                                            
2017-04-14 15:40:00 La oposición venezolana recobra el impulso con la protesta en la calle                     Dos semanas después de las sentencias del Tribunal Supremo de Justicia son cinco los mu...   145
                    El expresidente salvadoreño Funes, señalado por recibir 1,5 millones de dólares de Odeb... El exmandatario fue denunciado por un ex directivo de la constructora por recibir diner...   136
                    Columna | Por los clavos de Cristo                                                         Dónde está el pecado en decir que parecen excesivos los eventos religiosos estos días         88
                    España es el tercer país en pobreza infantil en la UE                                      Con un 40% de niños con  pobreza anclada , en la lista solo le superan Rumanía y Grecia       39
                    El pasajero evacuado por United Airlines demandará a la compañía                           No pueden tratarnos como ganado , afirma el abogado de David Dao, que perdió dos diente...    34

In [132]:
horas[16][['inter','title','desc', 'rank']].groupby(['inter','title','desc']).sum().sort_values(by='rank', ascending=False).head(5)

rank
inter               title                                                                                      desc                                                                                            
2017-04-14 16:00:00 La oposición venezolana recobra el impulso con la protesta en la calle                     Dos semanas después de las sentencias del Tribunal Supremo de Justicia son cinco los mu...   143
                    El expresidente salvadoreño Funes, señalado por recibir 1,5 millones de dólares de Odeb... El exmandatario fue denunciado por un ex directivo de la constructora por recibir diner...   111
                    Columna | Por los clavos de Cristo                                                         Dónde está el pecado en decir que parecen excesivos los eventos religiosos estos días        111
                    'Der Spiegel' afirma que Cristiano Ronaldo acordó pagar 258.000€ en 2010 para evitar un... El futbolista portugués Cristiano Ronaldo acordó pagar 375.000 dólares (258.172 euros d...    56
                    ‘Star Wars’ vuelve al lado oscuro con ‘Los últimos Jedi’                                   El primer avance del episodio VIII muestra el regreso de Luke Skywalker   Es el momento...    34

In [133]:
horas[17][['inter','title','desc', 'rank']].groupby(['inter','title','desc']).sum().sort_values(by='rank', ascending=False).head(5)

rank
inter               title                                                                                      desc                                                                                            
2017-04-14 16:20:00 'Der Spiegel' afirma que Cristiano Ronaldo acordó pagar 258.000€ en 2010 para evitar un... El futbolista portugués Cristiano Ronaldo acordó pagar 375.000 dólares (258.172 euros d...   140
                    La oposición venezolana recobra el impulso con la protesta en la calle                     Dos semanas después de las sentencias del Tribunal Supremo de Justicia son cinco los mu...   128
                    Columna | Por los clavos de Cristo                                                         Dónde está el pecado en decir que parecen excesivos los eventos religiosos estos días        108
                    El expresidente salvadoreño Funes, señalado por recibir 1,5 millones de dólares de Odeb... El exmandatario fue denunciado por un ex directivo de la constructora por recibir diner...    80
                    Acusan a policías y paramilitares de los últimos asesinatos en Venezuela                   La oposición venezolana ha regresado hoy a las calles de varios estados del país en su ...    52